# Overview

# Crawl data

In [1]:
#import
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd

In [5]:
#Crawl URL của 3000 sản phẩm và lưu vào file links.txt
def get_id(url):

    # Mở file để ghi
    with open('data/links.txt', 'w') as f:

        # Mỗi trang xe có link của 20 xe
        for i in range(1, 151):
            print(i)
            response = requests.get(url.format(i))

            # Tìm vị trí bắt đầu và kết thúc của chuỗi trong mã nguồn trang web
            start = response.text.find('"data":[{')
            end = response.text.find('"dataUpdateCount"', start)

            # Trích xuất phần cần thiết của mã nguồn trang web
            text = response.text[start:end]

            # Tìm tất cả các cặp "id" và "slug" trong phần đã trích xuất
            matches = re.findall(r'"id":"(\d{8})","slug":"([\w-]+)",', text)

            for match in matches:
                id, slug = match
                # Ghi vào file thay vì in ra
                f.write(f'https://carvago.com/car/{id}/{slug}\n')

# Sử dụng hàm với url đã được chỉnh sửa để chứa số trang
get_id('https://carvago.com/cars?page={}')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150


# Crawl thông tin của từng sản phẩm chi tiết

In [14]:
def extract_car_info(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            print('200\n')

            car_info_divs = soup.find_all('div', class_='css-1b3bvbl')  # Adjust the class if needed

            body_type = doors = seats = fuel = mileage = first_reg = interior = ex_color =power = consump = carname = " "
            special_feas = []

            for car_info_div in car_info_divs:
                attribute_name_div = car_info_div.find('div', class_='css-f6f0k1')
                if attribute_name_div is not None:
                    attribute_name = attribute_name_div.text.strip()

                    if attribute_name == 'Model':
                        car_link_div = car_info_div.find('div', class_='css-qres8i')
                        if car_link_div is not None:
                            car_link_tag = car_link_div.find('a')
                            if car_link_tag is not None:
                                car_link = car_link_tag['href']
                                carname = car_link.split('/')[2:]  # Extract the last part of the URL
                                carname = " ".join(carname)
                                print(carname)

                    # print("atribute name",attribute_name)
                    # print('-----')
                attribute_value_div = car_info_div.find('div', class_='css-qres8i')
                if attribute_value_div is not None:
                    attribute_value = attribute_value_div.text.strip()
                    # print("attri value:", attribute_value)
                    # print(attribute_value)
                    # Assign values based on attribute name
                    if attribute_name == 'Body':
                        body_type = attribute_value
                        # print("Body Type:", body_type)
                    elif attribute_name == 'Body color':
                        ex_color = attribute_value
                    elif attribute_name == 'Doors':
                        doors = attribute_value
                        # print("Doors:", doors)
                    elif attribute_name == 'Seats':
                        seats = attribute_value
                    elif attribute_name == 'Interior material':
                        interior = attribute_value
                    elif attribute_name == 'Fuel':
                        fuel = attribute_value
                    elif attribute_name == 'Mileage':
                        mileage = attribute_value
                    elif attribute_name == 'First registration':
                        first_reg = attribute_value
                    elif attribute_name == 'Power':
                        power = attribute_value
                    # elif attribute_name == 'Consumption':
                    #     consump = attribute_value

                        # print("Seats:", seats)

            # Print or use the extracted values as needed
                        
            print("Body Type:", body_type)
            print("Doors:", doors)
            print("Seats:", seats)
            print("Interior material",interior)
            print("Fuel",fuel)
            print("Mileage",mileage)
            print("First registration",first_reg)
            print("Body color: ", ex_color)
            print("Power: ",power)

            driver = webdriver.Chrome(ChromeDriverManager().install())
            driver.get(url)

        # Add this block to extract the total price
            price_div = driver.find_element_by_css_selector('[data-testid="page.detail.primaryPrice"]')
            # price_div = driver.find_element_by_css_selector('[data-testid="feature.transaction.receipt"]')
            # print(price_div.text)
            if price_div is not None:
                price_text = price_div.text
                # print(f'price_text: {price_text}')
                price = price_text.replace('\xa0', '')  # Remove the non-breaking spaces
                print("Total Price:", price)
            tags = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid="page.detail.tagsWrapper"] > div')
            for tag in tags:
                special_feas.append(tag.text)
            print(special_feas)
            driver.quit()
            

            return (carname, body_type, doors, seats, fuel, mileage, first_reg, interior, ex_color, power, price, special_feas)
    except requests.exceptions.RequestException as e:
        print(f"Failed to get the webpage at {url} due to error: {e}")
        return None, None, None, None, None, None, None, None, None, None, None, None


In [4]:
    

# Read links from data.txt


0 ----------------------
200
hyundai kona
Body Type: SUV / offroad
Doors: 4/5 doors
Seats: 5
Interior material Full leather interior
Fuel Electric
Mileage 10 km
First registration 9/2023
Body color:  Grey
Power:  160 kW
53 099 €
['Ventilated front seats', 'Heated rear seats', 'Parking assist system self-steering', 'Digital cockpit', 'Keyless entry', 'Electric adjustable front seats', 'Heated front seats', 'Head-up display', 'Apple CarPlay', 'Android auto']
1 ----------------------
200
volkswagen up!
Body Type: Hatchback
Doors: 4/5 doors
Seats: 4
Interior material Cloth interior
Fuel Petrol
Mileage 35 km
First registration 9/2023
Body color:  Black
Power:  61 kW
27 749 €
['Heated front seats', 'Bluetooth', 'Rain sensor', 'Hill-start assist', 'Multifunctional steering wheel', 'Side mirrors with electic adjustment', 'Daytime running lights', 'Leather steering wheel', 'Ambient lighting', 'USB']
2 ----------------------
200
seat ibiza
Body Type: Hatchback
Doors: 4/5 doors
Seats: 5
Interior 

# Lưu vào data/products.csv

In [15]:
df = pd.DataFrame(columns=['Car Name', 'Body Type', 'Doors', 'Seats', 'Fuel', 'Mileage', 'First Registration', 
                           'Interior Material', 'Exterior Color', 'Power', 'Price', 'Special Features'])

with open('data/links.txt', 'r') as file:
    links = file.read().splitlines()

# Loop through each link and extract information
i = 0
for link in links:
    car_info = extract_car_info(link)
    if (car_info is None):
        print(f"Skipping link due to error")
        continue  # Skip this link and move to the next one

    carname, body_type, doors, seats, fuel, mileage, first_reg, interior, ex_color, power, price, special_feas = car_info
    # Add the data to the DataFrame
    df.loc[i] = [carname, body_type, doors, seats, fuel, mileage, first_reg, interior, ex_color, power, price, special_feas]
    print(i,'----------------------')
    if i == 20:
        break
    i=i+1



Skipping link due to error
200

hyundai kona
Body Type: SUV / offroad
Doors: 4/5 doors
Seats: 5
Interior material Full leather interior
Fuel Electric
Mileage 10 km
First registration 9/2023
Body color:  Grey
Power:  160 kW
Total Price: 53 099 €
['Ventilated front seats', 'Heated rear seats', 'Parking assist system self-steering', 'Digital cockpit', 'Keyless entry', 'Electric adjustable front seats', 'Heated front seats', 'Head-up display', 'Apple CarPlay', 'Android auto']
0 ----------------------
200

volkswagen up!
Body Type: Hatchback
Doors: 4/5 doors
Seats: 4
Interior material Cloth interior
Fuel Petrol
Mileage 35 km
First registration 9/2023
Body color:  Black
Power:  61 kW


KeyboardInterrupt: 

In [9]:
df.to_csv("sell_data.csv",header=True)